In [1]:
#!/usr/bin/env python
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np, os, sys
from tqdm.notebook import tqdm    
from manipulations import get_classes, get_classes_from_header, get_Fs_from_header, load_challenge_data
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
N_CLASS = 27

In [3]:
MAX_RR = 500 # 60 beats/min => 60 beats/60 s ==> beat/1s ==> 500 samples / beat

In [4]:
from manipulations import get_abbr, get_name
from global_vars import labels, Dx_map, Dx_map_unscored
first_idx = {scored_code: None for scored_code in list(Dx_map['SNOMED CT Code'])}
first_idx_unscored = {unscored_code: None for unscored_code in list(Dx_map_unscored['SNOMED CT Code'])}

In [5]:
from global_vars import labels, normal_class, equivalent_mapping
normal_idx = np.argwhere(labels==int(normal_class))
def get_scored_class(code, labels):
    return [1 if label in code else 0 for label in labels]

In [6]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

if __name__ == '__main__':
    
    Datas = []
    Header_datas = []
    Classes = []
    Codes = []
    
    dataset_idx = {}
    dataset_data_labels = {}
    dataset_train_idx = {}
    dataset_test_idx = {}
    
    global_idx = 0
    datasets = [1,2,3,4,5,6]
    for dataset in datasets:
        print('Dataset ', dataset)
        # Parse arguments.
        if len(sys.argv) != 3:
            raise Exception('Include the input and output directories as arguments, e.g., python driver.py input output.')

        input_directory = '../NewData/{}/'.format(dataset)
        output_directory = '../Output/'

        # Find files.
        input_files = []
        for f in os.listdir(input_directory):
            if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
                input_files.append(f)

        if not os.path.isdir(output_directory):
            os.mkdir(output_directory)

        classes=get_classes(input_directory,input_files)

        num_files = len(input_files)
        datas = []
        header_datas = []
        dataset_data_labels[dataset] = []
        dataset_idx[dataset] = []
        for i, f in tqdm(enumerate(input_files)):
            #print('    {}/{}...'.format(i+1, num_files), f)
            tmp_input_file = os.path.join(input_directory,f)
            data,header_data = load_challenge_data(tmp_input_file)
            
            codes = get_classes_from_header(header_data)
            data_labels = get_scored_class(codes, labels)
            Codes.append(codes)
            
            datas.append(data[:,1000:7000])
            header_datas.append(header_data)
            dataset_data_labels[dataset].append(data_labels)
            dataset_idx[dataset].append(global_idx)
            global_idx += 1

        Datas += datas
        Header_datas += header_datas
        Classes += classes
        
        kf = MultilabelStratifiedKFold(5, random_state=0)
        train_idx, test_idx = next(kf.split(datas, np.array(dataset_data_labels[dataset])))


        dataset_train_idx[dataset] = train_idx +  dataset_idx[dataset][0]
        dataset_test_idx[dataset] = test_idx + dataset_idx[dataset][0]
        
        
        print('Done.')

Dataset  1


KeyboardInterrupt: 

In [22]:
# datasets = [1,2,3,4,5,6]
# all_train_idx2 = []
# for dataset in datasets:
#     all_train_idx2.extend(dataset_train_idx[dataset])
    
# all_test_idx2= []
# for dataset in datasets:
#     all_test_idx2.extend(dataset_test_idx[dataset])

In [ ]:
# with open('../saved/all_test_idx_stratified.pkl', 'wb') as all_test_idx_file:
#     pickle.dump(all_test_idx, all_test_idx_file)
# with open('../saved/all_train_idx_stratified.pkl', 'wb') as all_train_idx_file:
#     pickle.dump(all_train_idx, all_train_idx_file)

# Segmentation

In [8]:
# from signal_processing import myfilter, main_QRST
# Codes = []
# Q_locs = []
# idxes = []
# for idx in tqdm(range(0, len(Datas))): 
#     codes = get_classes_from_header(Header_datas[idx])
#     names = ', '.join([get_name(int(code), Dx_map, Dx_map_unscored) for code in codes])

#     filtered_Data = myfilter(Datas[idx], 500, vis=False)

#     # get the lead to apply Pan Tomkins
#     Q_loc = main_QRST(filtered_Data, idx, '', '', names, fig2=False)

#     # store
#     Codes.append(codes)
#     Q_locs.append(Q_loc)

#     idxes.append(idx)


In [9]:
# from signal_processing import myfilter, extract_QRST, main_QRST
# idx = 1511#37581#2932#6884 #6884 #6951 #227#8074 #325#158#325#34#17#8659#4#18#61#10927#6886# 8659
# codes = get_classes_from_header(Header_datas[idx])
# names = ', '.join([get_name(int(code), Dx_map, Dx_map_unscored) for code in codes])
# filtered_Data = myfilter(Datas[idx], 500, vis=False)
# #filtered_Data = myfilter(Datas[idx][:,-4000:-1000], 500, vis=False)
# Q_loc = main_QRST(filtered_Data, idx, '', '', names, fig2=False, verbose=True, vis=True)

In [10]:
# import pickle
# with open('../saved/newData_idxes_1000_7000_peakdist100.pkl', 'wb') as idxes_file:
#     pickle.dump(idxes, idxes_file)

# with open('../saved/newData_Codes_1000_7000_peakdist100.pkl', 'wb') as Codes_file:
#     pickle.dump(Codes, Codes_file)

# with open('../saved/newData_Q_locs_1000_7000_peakdist100.pkl', 'wb') as Q_locs_file:
#     pickle.dump(Q_locs, Q_locs_file)


In [8]:
import pickle
Q_locs = None
with open('../saved/newData_Q_locs_1000_7000_peakdist100.pkl', 'rb') as Q_locs_file:
    Q_locs = pickle.load(Q_locs_file)

Codes = None
with open('../saved/newData_Codes_1000_7000_peakdist100.pkl', 'rb') as Codes_file:
    Codes = pickle.load(Codes_file)

idxes = None
with open('../saved/newData_idxes_1000_7000_peakdist100.pkl', 'rb') as idxes_file:
    idxes = pickle.load(idxes_file)
    

In [9]:
import pickle
with open('../saved/all_train_idx_stratified.pkl', 'rb') as all_train_idx_file:
    all_train_idx = pickle.load(all_train_idx_file)
with open('../saved/all_test_idx_stratified.pkl', 'rb') as all_test_idx_file:
    all_test_idx = pickle.load(all_test_idx_file)

In [8]:
len(Codes)

43101

In [9]:
data_labels = np.array([get_scored_class(codes, labels) for codes in Codes])
print(data_labels.shape)

(43101, 27)


In [10]:
key_idxes = []
for key in equivalent_mapping.keys():
    print(key)
    key_idx = np.argwhere(labels==int(key)).flatten()[0]
    key_idxes.append(key_idx)
    val_idx = np.argwhere(labels==int(equivalent_mapping[key])).flatten()[0]
    key_pos = np.argwhere(data_labels[:,key_idx]==1).flatten()
    val_pos = np.argwhere(data_labels[:,val_idx]==1).flatten()
    data_labels[key_pos,val_idx] = 1
    data_labels[val_pos,key_idx] = 1

713427006
284470004
427172004


In [11]:
# non_empty_idx = np.argwhere(np.sum(data_labels, axis=1)!=0).flatten()
# empty_idx = np.argwhere(np.sum(data_labels, axis=1)==0).flatten()

In [12]:
X_train = []
Data_labels_train = []
Idxes_train = []
Idxes_dict_train = {}
ct = 0
for i in tqdm(all_train_idx):
    Q_loc = Q_locs[i]
    RR_avg = np.median([Q_loc[k+1] - Q_loc[k] for k in range(len(Q_loc)-1)])
    RR_th = (0.3 * RR_avg, 3 * RR_avg)
    
    ks = [k for k in range(len(Q_loc)-1) if Q_loc[k+1] - Q_loc[k] > RR_th[0] 
              and Q_loc[k+1] - Q_loc[k] < RR_th[1]]
    
    
    Idxes_dict_train[i] = []
    for k in ks:
        Data_labels_train.append(data_labels[i])
        X_train.append(Datas[i][:,Q_loc[k]:Q_loc[k+1]])
        Idxes_train.append(i)
        Idxes_dict_train[i].append(ct)
        
    ct += 1

/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
#X_test = []
Data_labels_test = []
Idxes_test = []
Idxes_dict_test = {}
ct = 0
for i in tqdm(all_test_idx):
        
    Q_loc = Q_locs[i]
    RR_avg = np.median([Q_loc[k+1] - Q_loc[k] for k in range(len(Q_loc)-1)])
    RR_th = (0.3 * RR_avg, 3 * RR_avg)
    
    ks = [k for k in range(len(Q_loc)-1) if Q_loc[k+1] - Q_loc[k] > RR_th[0] 
              and Q_loc[k+1] - Q_loc[k] < RR_th[1]]
    
    
    Idxes_dict_test[i] = []
    for k in ks:
        Data_labels_test.append(data_labels[i])
        #X_test.append(Datas[i][:,1000+Q_loc[k]:1000+Q_loc[k+1]])
        Idxes_test.append(i)
        Idxes_dict_test[i].append(ct)
        
    ct += 1

In [29]:
# Signals_train = np.zeros((len(X_train),12,MAX_RR))
# for i in range(len(X_train)):
#     Signals_train[i,:,:min(len(X_train[i][0]),MAX_RR)] = X_train[i][:,:MAX_RR]

In [30]:
# Signals_test = np.zeros((len(X_test),12,MAX_RR))
# for i in range(len(X_test)):
#     Signals_test[i,:,:min(len(X_test[i][0]),MAX_RR)] = X_test[i][:,:MAX_RR]

In [ ]:
# np.save('../saved/Signals_train_stratified.npy', Signals_train)
# np.save('../saved/Signals_test_stratified.npy', Signals_test)

In [17]:
# import pickle
# with open('../saved/Data_labels_train_stratified.pkl', 'wb') as Data_labels_train_file:
#     pickle.dump(Data_labels_train, Data_labels_train_file)

# with open('../saved/Data_labels_test_stratified.pkl', 'wb') as Data_labels_test_file:
#     pickle.dump(Data_labels_test, Data_labels_test_file)

# On y va

In [6]:
Signals_train = np.load('../saved/Signals_train_stratified.npy')
Signals_test = np.load('../saved/Signals_test_stratified.npy')

In [7]:
import pickle

with open('../saved/Data_labels_train_stratified.pkl', 'rb') as Data_labels_train_file:
    Data_labels_train = pickle.load(Data_labels_train_file)

with open('../saved/Data_labels_test_stratified.pkl', 'rb') as Data_labels_test_file:
    Data_labels_test = pickle.load(Data_labels_test_file)

In [64]:
run_name = 'ECGResNet_balanced_eqvclasses_no_posweight_no_weightedloss_500chn_n1_500'

In [65]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/{}'.format(run_name))


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [11]:
import torch
from torch import nn

In [12]:

def weighted_binary_cross_entropy2(sigmoid_x, y, weighted_matrix, weight=None, reduction=None):
    """
    Aha this is correct!
    sigmoid_x = nn.Sigmoid()(x)
    Args:
        sigmoid_x: predicted probability of size [N,C], N sample and C Class. Eg. Must be in range of [0,1]
        targets: true value, one-hot-like vector of size [N,C]
        pos_weight: Weight for postive sample
    """
    if not (y.size() == sigmoid_x.size()):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(y.size(), sigmoid_x.size()))
   
    #print("y.size(), sigmoid_x.size()", y.size(), sigmoid_x.size())
    sigmoid_x = torch.clamp(sigmoid_x,min=1e-7,max=1-1e-7) 
    loss = - torch.matmul(y*sigmoid_x.log() + (1-y)*(1-sigmoid_x).log(), weighted_matrix)
    
    if weight is not None:
        loss = loss * weight
        
    if reduction is None:
        return loss
    elif reduction == 'mean':
        return loss.mean()
    elif reduction == 'sum':
        return loss.sum()
    return None
    
class WeightedBCELoss(nn.Module):
    def __init__(self, weights, PosWeightIsDynamic= False, WeightIsDynamic= False, 
                 reduction='mean'):
        """
        Args:
            pos_weight = Weight for postive samples. Size [1,C]
            weight = Weight for Each class. Size [1,C]
            PosWeightIsDynamic: If True, the pos_weight is computed on each batch. If pos_weight is None, then it remains None.
            WeightIsDynamic: If True, the weight is computed on each batch. If weight is None, then it remains None.
        """
        super().__init__()

        self.register_buffer('weights', weights)
        self.reduction = reduction
        self.PosWeightIsDynamic = PosWeightIsDynamic

    def forward(self, input, target):
        if self.PosWeightIsDynamic:
            positive_counts = target.sum(dim=0)
            nBatch = len(target)
            self.pos_weight = (nBatch - positive_counts)/(positive_counts +1e-5)


        return weighted_binary_cross_entropy2(input, target,
                                             weighted_matrix=self.weights,
                                             reduction=self.reduction)

In [49]:
Signal_12leads_train = np.transpose(Signals_train, (1,0,2))
Signal_12leads_test= np.transpose(Signals_test, (1,0,2))

In [50]:
from torch.utils.data import Dataset
from torchvision import transforms

import os
class SignalDataset(Dataset):

    def __init__(self, signals, labels):
        self.signals = signals
        self.labels = labels

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample =(torch.cat([torch.Tensor(np.array([self.signals[idx,channel]])) for channel in range(12)], axis=0), 
                  torch.Tensor(self.labels[idx]))

        return sample
    
signal_datasets_train = SignalDataset(Signal_12leads_train[:,:,:MAX_RR], np.array(Data_labels_train)[:,:N_CLASS])
signal_datasets_test = SignalDataset(Signal_12leads_test[:,:,:MAX_RR], np.array(Data_labels_test)[:,:N_CLASS])


In [51]:
Signal_12leads_train.shape

(12, 321263, 750)

In [52]:
assert np.array(Data_labels_train).shape[1] == N_CLASS

In [53]:
assert signal_datasets_train[0][0].shape[1] == MAX_RR

In [54]:
from global_vars import weights

In [55]:
weights_tensor = torch.Tensor(weights).to(device)

In [35]:
from evaluation.evaluate_12ECG_score import compute_modified_confusion_matrix, compute_challenge_metric

In [36]:
def compute_score(labels, outputs, weights, normal_index=normal_idx):
    num_recordings, num_classes = np.shape(labels)
    # Compute the observed score.
    A = compute_modified_confusion_matrix(labels, outputs)
    observed_score = np.nansum(weights * A)

    # Compute the score for the model that always chooses the correct label(s).
    correct_outputs = labels
    A = compute_modified_confusion_matrix(labels, correct_outputs)
    correct_score = np.nansum(weights * A)

    # Compute the score for the model that always chooses the normal class.
    inactive_outputs = np.zeros((num_recordings, num_classes), dtype=np.bool)
    inactive_outputs[:, normal_index] = 1
    A = compute_modified_confusion_matrix(labels, inactive_outputs)
    inactive_score = np.nansum(weights * A)

    if correct_score != inactive_score:
        normalized_score = float(observed_score - inactive_score) / float(correct_score - inactive_score)
    else:
        normalized_score = float('nan')

    return normalized_score

In [37]:
def add_pr_curve_tensorboard(class_index, test_probs, test_preds, names, global_step, prefix):
    '''
    Takes in a "class_index" from 0 to 9 and plots the corresponding
    precision-recall curve
    '''

    writer.add_pr_curve(prefix + '_' + names[class_index],
                        test_preds[:,class_index],
                        test_probs[:,class_index],
                        global_step=global_step,
                        num_thresholds=127)
    writer.close()

In [66]:
import torch
dummy = torch.ones((10, MAX_RR, 12))

In [67]:
from resnet1d import ECGResNet
model = ECGResNet(MAX_RR, 27, n=1)

In [68]:
model(dummy).shape

torch.Size([10, 27])

In [69]:
from torchvision import datasets, models, transforms
from myeval import agg_y_preds_bags, binary_acc, geometry_loss
import torch.optim as optim
from torch.optim import lr_scheduler
from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import GroupKFold
import time
from imbalanced_sampler import ImbalancedDatasetSampler

st = time.time()
patience = 50
batch_size= 512#65000

saved_dir = '../saved/ECGResNet/'
model.to(device)



ECGResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv1d(500, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNet753Block(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv1dAuto(64, 64, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
                (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU(inplace=True)
              (2): Sequential(
                (0): Conv1dAuto(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
                (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [70]:
"""
Encode old label (multi-hot encoding) to new label (number) with a threshold (>=80-percentile length)
"""
Code_label_train = [get_scored_class(Data_labels_train[i], labels) for i in range(len(Data_labels_train)) ]


# key: multi-hot encoding label in str
# val: idx
labels_dict = {}
for idx, data_label in enumerate(Code_label_train):
    key = ''.join([str(l) for l in data_label])
    if key not in labels_dict:
        labels_dict[key] = [idx]
    else:
        labels_dict[key].append(idx)

# for each label (multi-hot encoding), their length
labels_dict_len = []
labels_dict_key = []
for key in labels_dict.keys():
    labels_dict_len.append(len(labels_dict[key]))
    labels_dict_key.append(key)
labels_dict_len = np.array(labels_dict_len)
labels_dict_key = np.array(labels_dict_key)

threshold = np.percentile(labels_dict_len, 80)

labels_dict_len_threshold = labels_dict_len[labels_dict_len>threshold]
labels_dict_key_threshold = labels_dict_key[labels_dict_len>threshold]

# old label (multi-hot encoding) => new label (number)
new_index_dict = {}
for i in range(len(labels_dict_key_threshold)):
    new_index_dict[labels_dict_key_threshold[i]] = i

# change label to new label
Code_label_train_new = np.zeros((len(Code_label_train),), dtype = int)
special_class = -1
for key, vals in labels_dict.items():
    for val in vals:
        if key not in labels_dict_key_threshold:
            Code_label_train_new[val] = special_class
        else:
            Code_label_train_new[val] = new_index_dict[key]
        

# distribution of classes in the dataset 
label_to_count = {}
for label in Code_label_train_new:
    if label in label_to_count:
        label_to_count[label] += 1
    else:
        label_to_count[label] = 1

# weight for each sample
sample_weights = [1.0 / label_to_count[label]
           for label in Code_label_train_new]
import torch
sample_weights = torch.DoubleTensor(sample_weights)

In [71]:
from imbalanced_sampler import  WeightedImbalancedDatasetSampler
trainDataset = torch.utils.data.Subset(signal_datasets_train, range(0,len(Signals_train), 1))
testDataset = torch.utils.data.Subset(signal_datasets_test, range(0,len(Signals_test), 1))

trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, pin_memory=True, 
                                       sampler=WeightedImbalancedDatasetSampler(trainDataset, weights=sample_weights))
testLoader = torch.utils.data.DataLoader(testDataset, batch_size=3000, shuffle = False, pin_memory=True)

In [72]:
#criterion_train = WeightedBCELoss(weights=weights_tensor, reduction='mean')
#criterion_test = WeightedBCELoss(weights=weights_tensor, reduction='mean')

class_weights_train = 1.0/np.sum(Data_labels_train, axis=0)
class_weights_train = class_weights_train / np.sum(class_weights_train)
class_weights_train = torch.Tensor(class_weights_train).to(device)


class_weights_test = 1.0/np.sum(Data_labels_test, axis=0)
class_weights_test = class_weights_test / np.sum(class_weights_test)
class_weights_test = torch.Tensor(class_weights_test).to(device)

pos_weight = np.ones(N_CLASS) * 5
pos_weight = torch.Tensor(pos_weight).to(device)

criterion_train = nn.BCEWithLogitsLoss(reduction='mean') 
criterion_test = nn.BCEWithLogitsLoss(reduction='mean') 

losses_train = []
losses_test = []

avg_losses_train = []
avg_losses_test = []





In [73]:
early_stopping = EarlyStopping(patience, verbose=False, 
                              saved_dir=saved_dir, 
                              save_name='resnet50_'+run_name)

In [74]:
optimizer = optim.Adam(model.parameters(), lr=0.01) 
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) #
# Decay LR by a factor of 0.1 every 100 epochs
#scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=20, mode='max')

In [75]:
names = [get_name(label, Dx_map, Dx_map_unscored) for label in labels]

assert len(labels) == 27

In [76]:
y_trains_tensor = None
y_tests_tensor = None
for epoch in range(0, 5000):

    model.train()

    running_loss_train = 0.0

    y_trains = [] # ground truth
    output_trains = [] # output
    for k, (X_train, y_train) in tqdm(enumerate(trainLoader)):
        y_train = y_train.to(device)
        X_train = X_train.to(device)
        optimizer.zero_grad()
        output_train = model(X_train)
        output_trains.append(output_train.cpu())
        
        loss_train = criterion_train(output_train, y_train)
        losses_train.append(loss_train.item())
        
        avg_loss_train = np.average(losses_train)
        avg_losses_train.append(avg_loss_train)
    
        
        if np.mod(k, 100) == 0:
            writer.add_scalar('training loss',
            avg_loss_train,
            epoch * (len(Data_labels_train)//batch_size//100+1) + k//100)
        
        y_trains.append(y_train.cpu())
            
        loss_train.backward()
        optimizer.step()
            
    #scheduler.step()
    

    y_tests = [] # ground truth
    output_tests = [] # output
    with torch.no_grad():
        model.eval()
        
        for X_test, y_test in testLoader:  
            y_test = y_test.to(device)
            X_test = X_test.to(device)
            output_test = model(X_test)

            loss_test = criterion_test(output_test, y_test)
            losses_test.append(loss_test.item())

            output_tests.append(output_test.cpu())
            y_tests.append(y_test.cpu())
            
            
            
        avg_loss_test = np.average(losses_test)
        avg_losses_test.append(avg_loss_test)

        writer.add_scalar('testing loss',
                avg_loss_test,
                epoch)



    y_trains_tensor = torch.cat(y_trains, axis=0) # ground truth
    y_tests_tensor = torch.cat(y_tests, axis=0) # ground truth

    output_trains = torch.cat(output_trains, axis=0) 
    y_train_preds = torch.sigmoid(output_trains)

    output_tests = torch.cat(output_tests, axis=0)
    y_test_preds = torch.sigmoid(output_tests)

    #output_trains = torch.cat(output_trains, axis=0)
#     y_train_preds_max, y_train_preds_mean, _ = agg_y_preds_bags(y_train_preds, bag_size=n_segments)
#     y_test_preds_max, y_test_preds_mean, _ = agg_y_preds_bags(y_test_preds, bag_size=n_segments)
#     _, _, y_trains = agg_y_preds_bags(y_trains, bag_size=n_segments)
#     _, _, y_tests = agg_y_preds_bags(y_tests, bag_size=n_segments)


    for class_idx in range(N_CLASS):
        add_pr_curve_tensorboard(class_idx, y_trains_tensor, y_train_preds, names, global_step=epoch, prefix='train')
        add_pr_curve_tensorboard(class_idx, y_tests_tensor, y_test_preds, names, global_step=epoch, prefix='test')



    acc, fmeasure, fbeta, gbeta = binary_acc(y_train_preds, y_trains_tensor)           
    acc2, fmeasure2, fbeta2, gbeta2 = binary_acc(y_test_preds, y_tests_tensor)
    geometry = geometry_loss(fbeta, gbeta)
    geometry2 = geometry_loss(fbeta2, gbeta2)
    
#     output_str = 'S{} {:.2f} min |\n Train Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f} |\n Valid Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f}\n '.format(
#         epoch, (time.time()-st)/60,
#         avg_loss_train, acc, fmeasure, fbeta, gbeta, geometry, np.nan,
#         avg_loss_test, acc2, fmeasure2, fbeta2, gbeta2, geometry2, np.nan)
#     print(output_str)
    score = compute_score(np.round(y_train_preds.data.numpy()), np.round(y_trains_tensor.data.numpy()), weights)
    score2 = compute_score(np.round(y_test_preds.data.numpy()), np.round(y_tests_tensor.data.numpy()), weights)
    output_str = 'S{} {:.2f} min |\n Train Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f} |\n Valid Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f}\n '.format(
        epoch, (time.time()-st)/60,
        avg_loss_train, acc, fmeasure, fbeta, gbeta, geometry, score,
        avg_loss_test, acc2, fmeasure2, fbeta2, gbeta2, geometry2, score2)
    scheduler.step(score2)
    
    writer.add_scalar('training score',
            score,
            epoch)

    writer.add_scalar('testing score',
            score2,
            epoch)
    
    print(output_str)

    with open(saved_dir+'loss_{}.txt'.format(run_name), 'a') as f:
        print(output_str, file=f)

    early_stopping(-score2, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break


#     output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure|Geomotry\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc2,auprc2,acc2,fmeasure2,fbeta2,gbeta2,geometry2)
#     print(output_string)     
#     with open(saved_dir+'score'+ str(i)+ '_epoch' + str(epoch) + '.txt', 'w') as f:
#         f.write(output_string)

#     avg_losses_train = np.array(avg_losses_train)
#     avg_losses_test = np.array(avg_losses_test)

#     np.save(saved_dir + 'avg_losses_train' + str(i) + '_epoch' + str(epoch), avg_losses_train)
#     np.save(saved_dir + 'avg_losses_test' + str(i) + '_epoch' + str(epoch), avg_losses_test)


IndexError: index 179976 is out of bounds for axis 0 with size 12

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
from global_vars import labels

cf_matrices = multilabel_confusion_matrix(y_trains_tensor.data.numpy(), np.round(y_train_preds.data.numpy()))

In [ ]:
for label, cf_matrix in zip(labels, cf_matrices):
    print(get_name(label, Dx_map, Dx_map_unscored))
    print(cf_matrix)
    print('\n')


In [ ]:
y_trains_tensor.data.numpy()